# Inference from any pose

With this notebook you can run inference of HOOD starting with a garment in any pose. For the body sequence you can use either:
* Sequence of SMPL parameters (see `$HOOD_DATA/fromanypose/pose_sequence.pkl` as an example) or
* Sequence of meshes (see `$HOOD_DATA/fromanypose/mesh_sequence.pkl` as an example)

You also need to have a garment mesh (as `.obj` file) which is aligned with the first frame of your body sequence.

In this notebook we first show how to convert a garment mesh in `.obj` format into a `.pkl` template used in HOOD. 

Then, we show how to use the configuration file `aux/from_any_pose.yaml` to run inference with over albitrary SMPL sequence and arbitrary mesh sequence.

Note that, pinned vertices are not yet supported in this example, so do not expect it to work with lower body garments (they would slide down :c)

## Set environmental variables

In [ ]:
import os

HOOD_PROJECT =  "/path/to/hood/repository"
HOOD_DATA = "/path/to/hood/data"

os.environ["HOOD_PROJECT"] = HOOD_PROJECT
os.environ["HOOD_DATA"] = HOOD_DATA

## Create template file from the .obj file:

Use `utils.mesh_creation::obj2template()` function to convert an `.obj` file into a template dictionary and then save it with `pickle_dump`

`$HOOD_DATA/fromanypose/tshirt.obj` is provided as an example

In [ ]:
import trimesh
from utils.mesh_creation import obj2template
from utils.common import pickle_load, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path

In [ ]:
# obj_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'tshirt.obj'
# out_template_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'tshirt.pkl'

obj_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'skirt.obj'
out_template_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'skirt.pkl'

template_dict = obj2template(obj_path, verbose=True)
pickle_dump(template_dict, out_template_path)

In [ ]:
from utils.mesh_creation import add_pinned_verts, add_pinned_verts_single_template


pinned_indices = \
[2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215,
 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235,
 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2327, 2328, 2351, 2492, 2497, 2669, 2670, 2671, 2674, 2837, 3139,
 3200, 3204, 3359, 3362, 3495, 3512, 3634, 3638, 3805, 3965, 3967, 4133, 4137, 4140, 4335, 4340, 4506, 4509, 4669, 4674,
 4749, 4812, 4849, 4853, 5138, 5309, 5342, 5469, 5474, 5503, 5646, 5650, 5654, 5855, 5857, 6028, 6091, 6204, 6209, 6280,
 6374, 6377, 6378, 6473, 6649, 6654, 6814, 6817, 6986, 6989, 6990, 6992, 7172, 7178, 7336, 7500, 7505, 7661, 7665, 7666]

add_pinned_verts_single_template(out_template_path, pinned_indices)

## Inference with a SMPL sequence

Here we use a sequence of SMPL parameters as a body sequence. The garment template needs to be aligned with the first frame of the sequence.

The SMPL pose sequence has to be a `.pkl` file containing a dictionary of the following items (see `$HOOD_DATA/fromanypose/pose_sequence.pkl` as an example):
* `body_pose`: np.array [N, 69]
* `global_orient`: np.array [N, 3]
* `transl`: np.array [N, 3]
* `betas`: np.array [10,]

### Load runner from `from_any_pose` config

In [ ]:
from utils.validation import Config as ValidationConfig
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch
from utils.arguments import create_runner

In [ ]:
modules, config = load_params('aux/from_any_pose')

runner_module, runner, aux_modules = create_runner(modules, config)

checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'postcvpr.pth'
state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

### Create a data loader

To do that, first, set several variables to specify which garment and which pose sequence you want to use:

- `pose_sequence_type`: type of the pose sequence can be either `smpl` or `mesh`. For this example, set `smpl`
- `pose_sequence_path` path to the smpl pose sequence relative to `$HOOD_DATA`. For this example we use `fromanypose/pose_sequence.pkl`
- `garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/tshirt.pkl`
- `smpl_model` path to the SMPL model you want to use relative to `$HOOD_DATA`.  For this example we use `smpl/SMPL_FEMALE.pkl`


Then, we create a dataloader with `make_fromanypose_dataloader()` function (see its code for more details)

In [ ]:
from utils.datasets import make_fromanypose_dataloader

pose_sequence_type = 'smpl'
pose_sequence_path = Path('fromanypose') / 'pose_sequence.pkl'
# garment_template_path = Path('fromanypose') / 'tshirt.pkl'
garment_template_path = Path('fromanypose') / 'skirt.pkl'
smpl_model = 'smpl/SMPL_FEMALE.pkl'

dataloader = make_fromanypose_dataloader(pose_sequence_type=pose_sequence_type, 
                       pose_sequence_path=pose_sequence_path, 
                       garment_template_path=garment_template_path, 
                       smpl_model=smpl_model)

### load sample, infer and save trajectories

To visualise saved trajectories, see `Inference.ipynb::write_video`

In [ ]:
sample = next(iter(dataloader))

trajectories_dict = runner.valid_rollout(sample)
# Save the sequence to disc
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_f.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

## Inference with a mesh sequence

Here we use a sequence of arbitrary meshes.  The garment template needs to be aligned with the first frame of the sequence.

The mesh sequence has to be a `.pkl` file containing a dictionary of the following items (see `$HOOD_DATA/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, 3]
* `faces`: np.array [F, 3]

### Load runner and dataloader from `from_any_pose` config

In [ ]:
from utils.validation import Config as ValidationConfig
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch

In [ ]:
modules, config = load_params('aux/from_any_pose')


runner_module, runner, aux_modules = create_runner(modules, config)

checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'postcvpr.pth'
state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

### Create a data loader

Same as in the previous part, set several variables to specify which garment and which pose sequence you want to use:

- `pose_sequence_type`: type of the pose sequence can be either `smpl` or `mesh`. For this example, set `mesh`
- `pose_sequence_path` path to the smpl pose sequence relative to `$HOOD_DATA`. For this example we use `fromanypose/mesh_sequence.pkl`
- `garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/tshirt.pkl`

In [ ]:
pose_sequence_type = 'mesh'
pose_sequence_path = Path('fromanypose') / 'mesh_sequence.pkl'
# garment_template_path = Path('fromanypose') / 'tshirt.pkl'
garment_template_path = Path('fromanypose') / 'skirt.pkl'


dataloader = make_fromanypose_dataloader(pose_sequence_type=pose_sequence_type, 
                       pose_sequence_path=pose_sequence_path, 
                       garment_template_path=garment_template_path)

### load sample, infer and save trajectories

To visualise saved trajectories, see `Inference.ipynb::write_video`

In [ ]:
sample = next(iter(dataloader))

trajectories_dict = runner.valid_rollout(sample)
# Save the sequence to disc
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_m.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)